# 02 — Basic Clustering (DBSCAN‑like)
In pure Pyodide environments, `scikit-learn` may not be present.
We'll implement a **very small** DBSCAN‑like routine with NumPy for teaching.

In [ ]:
import numpy as np
import plotly.graph_objects as go

pts = np.load("data/points_small.npz")["points"]

# --- Minimal DBSCAN‑ish (radius graph + core expansion); for teaching only ---
def dbscan_numpy(X, eps=0.15, min_pts=12):
    N = len(X); labels = -np.ones(N, dtype=int)
    visited = np.zeros(N, dtype=bool)
    cluster_id = 0

    # precompute simple radius neighbors (O(N^2) for tiny sets)
    d2 = ((X[:,None,:]-X[None,:,:])**2).sum(axis=2)
    rad = d2 <= (eps**2)

    for i in range(N):
        if visited[i]: 
            continue
        visited[i] = True
        neighbors = np.where(rad[i])[0]
        if neighbors.size < min_pts:
            labels[i] = -1  # noise
        else:
            # start new cluster
            labels[i] = cluster_id
            seeds = set(neighbors.tolist())
            seeds.discard(i)
            while seeds:
                j = seeds.pop()
                if not visited[j]:
                    visited[j] = True
                    nbs = np.where(rad[j])[0]
                    if nbs.size >= min_pts:
                        seeds.update(set(nbs.tolist()))
                if labels[j] == -1:
                    labels[j] = cluster_id
                if labels[j] == -1 or labels[j] == -1:  # keep assigned
                    labels[j] = cluster_id
            cluster_id += 1
    return labels

labels = dbscan_numpy(pts, eps=0.18, min_pts=15)
n_clusters = labels.max() + 1
n_noise = (labels == -1).sum()
print(f"clusters: {n_clusters}, noise: {n_noise}")

# visualize
# map labels -> scalar colors (noise -> -1)
colors = labels.astype(float)
fig = go.Figure(data=[go.Scatter3d(
    x=pts[:,0], y=pts[:,1], z=pts[:,2],
    mode="markers",
    marker=dict(size=2, color=colors)
)])
fig.update_layout(height=500, margin=dict(l=0,r=0,b=0,t=30), title="DBSCAN‑like clustering (NumPy)")
fig.show()